In [1]:
# Importing the necessary libraries.
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import requests

In [2]:
# Using an API to fetch the data from EODHD's financial data.
def get_stock_data(symbol, start_date, end_date, api_key):
    url = f"https://eodhistoricaldata.com/api/eod/{symbol}"
    params = {
        "from": start_date,
        "to": end_date,
        "api_token": api_key,
        "fmt": "json"
    }
    response = requests.get(url, params)
    if response.status_code == 200:
        data = pd.DataFrame(response.json())
        data['date'] = pd.to_datetime(data['date'])
        return data.set_index('date')
    else:
        raise Exception(f"APIS request failed with status code {response.status_code}")
        


In [3]:
# Preparing the data
def prepare_data(df, look_back=30):
    X, y = [], []
    for i in range(len(df) - look_back):
        X.append(df.iloc[i:i+look_back]['close'].values)
        y.append(df.iloc[i+look_back]['close'])
    return np.array(X), np.array(y)

In [4]:
# Loading the data
api_key = "demo"
apple_data = get_stock_data("AAPL", "2024-01-01", "2024-09-31", api_key)
print(apple_data.head())

              open    high     low   close  adjusted_close    volume
date                                                                
2024-01-02  187.15  188.44  183.89  185.64        184.0815  82488700
2024-01-03  184.22  185.88  183.43  184.25        182.7032  58414500
2024-01-04  182.15  183.09  180.88  181.91        180.3828  71983600
2024-01-05  181.99  182.76  180.17  181.18        179.6589  62379700
2024-01-08  182.09  185.60  181.50  185.56        184.0022  59144500


In [5]:
# Breaking the data into training and testing data.
X, y = prepare_data(apple_data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print(f"Model Score: {model.score(X_test, y_test)}")

Model Score: 0.9737878392536558
